# Crashes to JSON

Use `Data Query Tool` to create a new export from UCONN Crashes
https://www.ctcrash.uconn.edu/QueryTool2.action

* Since the tool does not let you export large datasets (over 90,000 persons), export multiple times for different time periods and specify all export IDs in the `exports` list below.

This notebook combines data from two tables (0 and 2) into a small JSON file and a CSV file (same content, different formats), which is an array of objects that look like this:
```js
{
    "id": 770826,    // crash id (used to pull the diagram)
    "x": 41.73936,   // latitude
    "y": -72.66404,  // longitude
    "k": 1/1/2020    ///date for crash 
    "d": 1593820800, // timestamp
    "t": 09:41:00    // time of crash
    "s": 0,          //// Most Sever Injury
    "r": 1,          // route class (1=Interstate, 2=USRoute, 3=State, 4=Local)
    "p": 0,          // involves a pedestrian or other pedestrian? (1=true, 0=false)
    "c": 0,          // involves a cyclist or other cyclist? (1=true, 0=false)
    "to": 0          /// total other injuries (minor) 
    "ts": 0          /// total serious injuries
    "tf": 0          /// total fatal injuries
```

Note: the timestamp `d` needs to be multiplied by `100,000` for a proper JS timestamp.

In [1]:
# Importing in Panda Library used for data manipulation and analysis
import pandas as pd

In [2]:
# Setting name of folder that raw data is in
exports = ['40996']

### Import Injury Statuses per Crash Id from table 2

In [3]:
# This data frame is used for calculating the total injury status types for a given crash id
total_dfs = []

# Iterating through export_xxxxx_2.csv (roles) file and creates new array of data only from CrashID and Injury Status
for export in exports:
    df = pd.read_csv(
        f'export_{export}/export_{export}_2.csv',
        skiprows=1,
        usecols=['CrashId', 'Injury Status'],
        encoding='ISO-8859-1'
    )    
    
    
    total_dfs.append( df ) # Adding two columns of data to new array 

totals = pd.concat(total_dfs).groupby(['CrashId'])['Injury Status'].unique() # Grouping data using two columns specified

### Calculating Total "Other Injuries" per Crash Id

In [4]:
currentId = df['CrashId'][0] # setting first id in the data
col1 = [] # List to put ids in
col2 = [] # List to put totals in 
countO = 0
# Loop that iterates over the ids and checks if codes that fall into "Other Injuries" are present 
# Total of other injuries is then summed 
for ind in df.index:
    if df['CrashId'][ind] == currentId:
        if df['Injury Status'][ind] == "O" or df['Injury Status'][ind] == "B" or df['Injury Status'][ind] == "C": # Checking injury status for codes 
            countO = countO +  1  # incrementing total for current id 
    else:
    # Once the id changes the last id is put into the list as well as the total that is calculated
        col1.append(df['CrashId'][ind - 1]) # Adding previous id to id list
        col2.append(countO) # Adding totals summed for that id to totals list
        currentId = df['CrashId'][ind] # Setting current id to the next id once switch of id has occurred
        if df['Injury Status'][ind] == "O":
            countO = 0 # setting count back to zero once id has switched
            countO = countO +  1 # incrementing total for current id
        else:
            countO = 0 # setting count back to zero 
d = {'id': col1, 'to': col2 } # adding list of unique ids and totals for id together
totalsO = pd.DataFrame(data=d) # creating a data frame for tow columns 
print(totalsO)

            id  to
0         2000   2
1         2001   1
2         2002   1
3         2003   2
4         2004   3
...        ...  ..
13923  1014560   2
13924  1014704   2
13925  1014706   4
13926  1015010   1
13927  1015532   0

[13928 rows x 2 columns]


### Calculating Total "Serious Injuries" per Crash Id

In [5]:
currentId = df['CrashId'][0]
col1 = [] # list to store ids
col2 = [] # list to store totals 
countS = 0 # used to increment and keep track of each id total 

# loop that sums total amount of "A codes" per unique id 
for ind in df.index:
    if df['CrashId'][ind] == currentId:
        if df['Injury Status'][ind] == "A":
            countS = countS +  1
    else:
        # once crash id switches data for previous id must be stored 
        col1.append(df['CrashId'][ind - 1])
        col2.append(countS)
        currentId = df['CrashId'][ind]
        if df['Injury Status'][ind] == "A":
            countS = 0
            countS = countS +  1
        else:
            countS = 0
d = {'id': col1, 'ts': col2 } # adding two lists together of ids and totals 
totalsS = pd.DataFrame(data=d)

### Calculating Total "Fatal Injuries" per Crash Id

In [6]:
currentId = df['CrashId'][0]
col1 = [] # list to store ids
col2 = []  # list to store totals
countK = 0
# loop that sums total amount of "A codes" per unique id 
for ind in df.index:
    if df['CrashId'][ind] == currentId:
        if df['Injury Status'][ind] == "K":
            countK = countK +  1
 # once crash id switches data for previous id is stored and total for that id is stored 
    else:
        col1.append(df['CrashId'][ind - 1])
        col2.append(countK)
        currentId = df['CrashId'][ind]
        if df['Injury Status'][ind] == "K":
            countK = 0
            countK = countK +  1
        else:
            countK = 0
d = {'id': col1, 'tf': col2 }
totalsK = pd.DataFrame(data=d)


### Importing Person Type Data from table 2

In [7]:
roles_dfs = []

# Iterating through export_xxxxx_2.csv (roles) file and creates new array of data only from CrashID and Person Type
for export in exports:
    df = pd.read_csv(
        f'export_{export}/export_{export}_2.csv',
        skiprows=1,
        usecols=['CrashId', 'Person Type'],
        encoding='ISO-8859-1'
    )    
    roles_dfs.append( df ) # Adding two columns of data to new array 

roles = pd.concat(roles_dfs).groupby(['CrashId'])['Person Type'].unique()# Grouping data using two columns specified

In [8]:
roles.head(10)

CrashId
2000       [1]
2001       [1]
2002       [1]
2003       [1]
2004    [1, 2]
2005    [9, 1]
2006       [1]
2007       [1]
2008    [2, 1]
2009    [2, 1]
Name: Person Type, dtype: object

### Import crashes from table 0

In [9]:
crashes_dfs = []

# Iterating through export_xxxxx_0.csv (crashes) file and creates new array of data using following categories:
#'CrashId', 'Latitude', 'Longitude', 'Date Of Crash', 'Time of Crash', 'Most Severity', 'Route Class'
for export in exports:
    df = pd.read_csv(
        f'export_{export}/export_{export}_0.csv',
        skiprows=1,
        usecols=['CrashId', 'Latitude', 'Longitude', 'Date Of Crash',
                 'Time of Crash', 'Most Severe Injury', 'Route Class']
    )
    df['Date Place Holder'] = df.loc[:, 'Date Of Crash'] # Adding duplicate column of date as last column to be used for new CSV
    df = df[['CrashId', 'Latitude', 'Longitude', 'Date Of Crash', 'Date Place Holder',
                 'Time of Crash', 'Most Severe Injury', 'Route Class']] # Rearranging Columns so that dates are next to eachother
    crashes_dfs.append( df ) 
    
   
    
crashes = pd.concat(crashes_dfs) # Grouping data using 8 columns specified 

### Merge data and save to .json and .csv

In [10]:
crashes.head() 

,CrashId,Latitude,Longitude,Date Of Crash,Date Place Holder,Time of Crash,Most Severe Injury,Route Class
0,2000,41.754402,-72.736591,2015-01-02,2015-01-02,09:41:00,O,4.0
1,2001,41.724997,-72.759667,2015-01-01,2015-01-01,18:00:00,O,NaN
2,2002,41.724880,-72.740740,2015-01-03,2015-01-03,09:42:00,O,4.0
3,2003,41.762320,-72.740570,2015-01-03,2015-01-03,12:05:00,O,4.0
4,2004,41.752930,-72.743672,2015-01-03,2015-01-03,14:49:00,O,4.0


### Filter Data From Exports 0 and 2 to create Smaller CSV and JSON File

In [11]:
# Rename columns to make JSON output smaller
crashes.columns = ['id', 'x', 'y', 'k', 'd', 't', 's', 'r']

# Transforming datestamp into timestap 
crashes.d = crashes.d.apply(lambda x: int(pd.to_datetime(x).timestamp() / 100))
crashes.t = crashes.t.apply(lambda x: x[:-3])

# Setting up pedestrian and cyclist flags to CSV, adds a 1(true) or 0(false) for pedestrain or cyclist column 
# 3 is person type code for Pedestrain, 4 is Other Pedestrain 
# 5 is person type code for Cyclist, 100 is placeholder for when we find Other Cyclist Code 
crashes['p'] = crashes.id.apply(lambda x: 1 if x in roles and ( 3 in roles[x] or 4 in roles[x]) else 0) 
crashes['c'] = crashes.id.apply(lambda x: 1 if x in roles and ( 5 in roles[x] or 100 in roles[x]) else 0)

# Joining the three data frames that contain id and totals for injury status (other, serious and fatal). 
crashes_update1 = pd.merge(crashes, totalsO, how='inner', on = 'id') # Joining Other Injury Totals with Crash Data 
crashes_update2 = pd.merge(crashes_update1, totalsS, how='inner', on = 'id') # Joining Serious Injury Totals with Crash Data
crashes_update3 = pd.merge(crashes_update2, totalsK, how='inner', on = 'id')  # Joining Fatal Injury Totals with Crash Data



In [12]:
# Save as JSON
crashes_update3.to_json('./crashes.json', orient='records', double_precision=5)

# Save as CSV
crashes_update3.to_csv('crashes.csv', index=False)

In [13]:
crashes_update3.head() 

,id,x,y,k,d,t,s,r,p,c,to,ts,tf
0,2000,41.754402,-72.736591,2015-01-02,14201568,09:41,O,4.0,0,0,2,0,0
1,2001,41.724997,-72.759667,2015-01-01,14200704,18:00,O,NaN,0,0,1,0,0
2,2002,41.724880,-72.740740,2015-01-03,14202432,09:42,O,4.0,0,0,1,0,0
3,2003,41.762320,-72.740570,2015-01-03,14202432,12:05,O,4.0,0,0,2,0,0
4,2004,41.752930,-72.743672,2015-01-03,14202432,14:49,O,4.0,0,0,3,0,0


### Just custom calculations – not needed for processing